# Great Caria: Topological Fragility Engine

**Goal**: Detect systemic fragility using Topological Data Analysis.

Core insight: Before crises, market structure **simplifies**. Correlations
collapse, the "sponge" becomes a "rod". TDA detects this geometrically.

In [ ]:
!pip install gudhi numpy pandas scipy matplotlib seaborn yfinance -q

import numpy as np
import pandas as pd
import gudhi
from scipy.spatial.distance import pdist, squareform
from scipy.sparse.csgraph import minimum_spanning_tree
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

print(f"GUDHI version: {gudhi.__version__}")

## 1. Load Market Data

We need daily returns for the correlation-based distance metric.

In [ ]:
import yfinance as yf

# Global indices representing major economies
INDICES = {
    'USA': '^GSPC',     # S&P 500
    'CHN': '000001.SS', # Shanghai Composite
    'JPN': '^N225',     # Nikkei 225
    'DEU': '^GDAXI',    # DAX
    'GBR': '^FTSE',     # FTSE 100
    'FRA': '^FCHI',     # CAC 40
    'IND': '^BSESN',    # BSE SENSEX
    'BRA': '^BVSP',     # Bovespa
    'CAN': '^GSPTSE',   # TSX Composite
    'KOR': '^KS11',     # KOSPI
    'AUS': '^AXJO',     # ASX 200
    'MEX': '^MXX',      # IPC Mexico
}

def load_index_returns(indices, start='2000-01-01'):
    """Load daily returns for global indices."""
    returns = {}
    
    for name, ticker in tqdm(indices.items(), desc="Loading indices"):
        try:
            df = yf.download(ticker, start=start, progress=False)
            if len(df) > 0:
                returns[name] = df['Close'].pct_change().dropna()
        except Exception as e:
            print(f"Warning: {name} failed: {e}")
    
    # Align all series
    df_returns = pd.DataFrame(returns)
    df_returns = df_returns.dropna()
    
    print(f"Loaded {len(df_returns)} days, {len(df_returns.columns)} countries")
    return df_returns

returns_df = load_index_returns(INDICES)

## 2. Persistence Diagram Computation

In [ ]:
def compute_distance_matrix(returns, window=60):
    """
    Compute correlation-based distance matrix.
    d(x,y) = sqrt(2 * (1 - correlation(x,y)))
    """
    if isinstance(returns, pd.DataFrame):
        corr = returns.corr().values
    else:
        corr = np.corrcoef(returns.T)
    
    # Handle numerical issues
    corr = np.clip(corr, -1, 1)
    dist = np.sqrt(2 * (1 - corr))
    np.fill_diagonal(dist, 0)
    
    return dist

def compute_persistence_diagram(dist_matrix, max_edge=2.0):
    """
    Compute persistence diagram using Rips complex.
    
    Returns:
        diag: List of (dimension, (birth, death)) tuples
    """
    rips = gudhi.RipsComplex(distance_matrix=dist_matrix, max_edge_length=max_edge)
    st = rips.create_simplex_tree(max_dimension=2)
    diag = st.persistence()
    
    return diag

def compute_fragility_score(diag):
    """
    Compute fragility score from persistence diagram.
    
    Fragility = 1 - complexity
    Low Betti-1 (few loops) = simple structure = high fragility
    High Betti-1 (many loops) = complex structure = low fragility
    """
    # Count persistent features in dimension 1 (loops)
    h1_features = []
    total_persistence = 0
    
    for dim, (birth, death) in diag:
        if dim == 1:  # 1-dimensional holes (loops)
            if death == float('inf'):
                life = 2.0 - birth  # Assume max edge = 2.0
            else:
                life = death - birth
            
            if life > 0.1:  # Filter noise
                h1_features.append(life)
                total_persistence += life
    
    # Complexity score: normalized total persistence
    complexity = min(total_persistence * 10, 100)
    
    # Fragility is inverse of complexity
    fragility = 100 - complexity
    
    return {
        'fragility': fragility,
        'complexity': complexity,
        'n_loops': len(h1_features),
        'total_persistence': total_persistence
    }

# Test on recent data
recent_returns = returns_df.tail(60)
dist_matrix = compute_distance_matrix(recent_returns)
diag = compute_persistence_diagram(dist_matrix)
score = compute_fragility_score(diag)

print(f"\nCurrent Fragility Score: {score['fragility']:.1f}%")
print(f"Complexity: {score['complexity']:.1f}")
print(f"Persistent Loops: {score['n_loops']}")

## 3. Rolling Fragility Signal (250-day early warning)

In [ ]:
def compute_rolling_fragility(returns_df, window=60, step=5):
    """
    Compute rolling fragility score over time.
    
    This is the key early warning indicator:
    Rising fragility = market structure simplifying = crisis approaching
    """
    dates = []
    fragilities = []
    complexities = []
    
    for i in tqdm(range(window, len(returns_df), step), desc="Computing fragility"):
        window_returns = returns_df.iloc[i-window:i]
        
        try:
            dist = compute_distance_matrix(window_returns)
            diag = compute_persistence_diagram(dist)
            score = compute_fragility_score(diag)
            
            dates.append(returns_df.index[i])
            fragilities.append(score['fragility'])
            complexities.append(score['complexity'])
        except Exception as e:
            pass
    
    result = pd.DataFrame({
        'date': dates,
        'fragility': fragilities,
        'complexity': complexities
    }).set_index('date')
    
    return result

# Compute rolling fragility
fragility_series = compute_rolling_fragility(returns_df, window=60, step=5)

print(f"\nFragility series: {len(fragility_series)} observations")
print(f"Date range: {fragility_series.index.min()} to {fragility_series.index.max()}")

## 4. Visualize: Fragility vs Crisis Events

In [ ]:
# Known crisis dates
CRISIS_EVENTS = {
    '2000-03-10': 'Dot-Com Peak',
    '2007-10-09': 'Pre-GFC Peak',
    '2008-09-15': 'Lehman Collapse',
    '2020-02-19': 'COVID Peak',
    '2022-01-03': 'Rate Hike Cycle'
}

fig, axes = plt.subplots(2, 1, figsize=(16, 10), sharex=True)

# Fragility
ax1 = axes[0]
ax1.fill_between(fragility_series.index, fragility_series['fragility'], 
                 alpha=0.3, color='red')
ax1.plot(fragility_series.index, fragility_series['fragility'], 
         color='red', linewidth=1.5, label='Fragility')

# Mark crisis events
for date, name in CRISIS_EVENTS.items():
    try:
        ax1.axvline(pd.Timestamp(date), color='black', linestyle='--', alpha=0.7)
        ax1.text(pd.Timestamp(date), ax1.get_ylim()[1]*0.95, name, 
                rotation=45, fontsize=8, ha='right')
    except:
        pass

ax1.set_ylabel('Fragility Score', fontsize=12)
ax1.set_title('Great Caria: Topological Fragility Early Warning Signal', fontsize=14)
ax1.axhline(70, color='orange', linestyle=':', label='Warning Threshold (70)')
ax1.axhline(85, color='red', linestyle=':', label='Critical Threshold (85)')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Market index (S&P 500 as reference)
ax2 = axes[1]
if 'USA' in returns_df.columns:
    cumulative = (1 + returns_df['USA']).cumprod()
    ax2.plot(cumulative.index, cumulative, color='blue', linewidth=1.5)
    ax2.set_ylabel('S&P 500 (Cumulative)', fontsize=12)
    ax2.set_yscale('log')
    
    # Mark crisis events
    for date, name in CRISIS_EVENTS.items():
        try:
            ax2.axvline(pd.Timestamp(date), color='black', linestyle='--', alpha=0.7)
        except:
            pass

ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('great_caria_fragility_history.png', dpi=150)
plt.show()

## 5. Export Fragility Signal

In [ ]:
import json

# Current fragility status
current = fragility_series.iloc[-1]

# Determine status
if current['fragility'] >= 85:
    status = 'CRITICAL'
    color = 'red'
elif current['fragility'] >= 70:
    status = 'WARNING'
    color = 'orange'
elif current['fragility'] >= 50:
    status = 'ELEVATED'
    color = 'yellow'
else:
    status = 'STABLE'
    color = 'green'

fragility_output = {
    'timestamp': pd.Timestamp.now().isoformat(),
    'current': {
        'fragility': float(current['fragility']),
        'complexity': float(current['complexity']),
        'status': status,
        'color': color
    },
    'history': {
        'dates': [d.isoformat() for d in fragility_series.index[-30:]],
        'fragility': fragility_series['fragility'].tail(30).tolist()
    },
    'interpretation': {
        'CRITICAL': 'Market topology collapsed. Correlation approaching 1. Systemic risk extreme.',
        'WARNING': 'Structure simplifying. Diversification failing. Heightened fragility.',
        'ELEVATED': 'Some stress detected. Monitor closely.',
        'STABLE': 'Complex, sponge-like topology. Healthy market structure.'
    }
}

with open('great_caria_fragility.json', 'w') as f:
    json.dump(fragility_output, f, indent=2)

print(f"\n" + "="*50)
print(f"GREAT CARIA FRAGILITY STATUS: {status}")
print(f"="*50)
print(f"Fragility Score: {current['fragility']:.1f}%")
print(f"Complexity: {current['complexity']:.1f}")
print(f"\nExported to great_caria_fragility.json")